In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Datei-Pfad (im gleichen Ordner wie das Notebook oder anpassen)
FILE_PATH = "../data/raw/Stoerliste_Heckrungenanlage_2023_NEU.xlsx"
SHEET_NAME = "Aufschreibung"

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

In [ ]:
# --- Laden ---
df_raw = pd.read_excel(FILE_PATH, sheet_name=SHEET_NAME)

print("Rohdaten:", df_raw.shape)
display(df_raw.head(3))


In [ ]:
# --- Spaltennamen normalisieren (Zeilenumbrüche/Mehrfachspaces entfernen) ---
df = df_raw.copy()
df.columns = [re.sub(r"\s+", " ", str(c).strip()) for c in df.columns]

# Zielspalte finden (robust, falls in Excel Zeilenumbrüche/Spaces anders sind)
target_pattern = re.compile(r"^Dauer\s*Org-?Mangel$", re.IGNORECASE)
start_col = None
for c in df.columns:
    if target_pattern.match(c):
        start_col = c
        break

if start_col is None:
    # Fallback: suche nach beiden Wörtern
    candidates = [c for c in df.columns if ("Dauer" in c) and ("Org" in c) and ("Mangel" in c)]
    if candidates:
        start_col = candidates[0]

if start_col is None:
    raise ValueError("Spalte 'Dauer Org-Mangel' konnte nicht gefunden werden. Bitte Spaltennamen prüfen.")

start_idx = list(df.columns).index(start_col)
cols_from = list(df.columns)[start_idx:]

print("Startspalte:", start_col)
print("Spalten ab Startspalte:", cols_from)


In [ ]:
# --- Leere Zeilen entfernen: wenn ab Startspalte (inkl.) ALLES leer ist ---
df_clean = df.copy()

# Leere Strings -> NA (nur in object/string-Spalten)
for c in cols_from:
    if df_clean[c].dtype == object:
        df_clean[c] = df_clean[c].astype("string").str.strip()
        df_clean.loc[df_clean[c].isin(["", "nan", "NaN"]), c] = pd.NA

mask_keep = df_clean[cols_from].notna().any(axis=1)
df_clean = df_clean.loc[mask_keep].copy()

print("Nach dem Entfernen leerer Zeilen:", df_clean.shape)
display(df_clean.head(5))


In [ ]:
# --- Alle Varianten aus "Unterbrechungsursache" alphabetisch sammeln ---

# "Unterbrechungsursache" bereinigen: Zeilenumbrüche/Mehrfachspaces -> 1 Space, trimmen
bem = df_clean["Unterbrechungsursache"].astype("string")
bem_norm = (
    bem
    .astype("string")
    .str.lower()
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

# NaNs sichtbar machen (sonst gehen sie ggf. unter)
bem_norm = bem_norm.fillna("<NA>")

# Alphabetische Liste + Counts
bem_counts = (
    bem_norm
    .value_counts(dropna=False)
    .sort_index()  # alphabetisch nach dem Wert
    .reset_index()
    .rename(columns={"index": "Unterbrechungsursache", "count": "Anzahl"})
)

display(bem_counts)
print(f"Anzahl verschiedener Varianten: {bem_counts.shape[0]}")

In [ ]:
# --- Alle Varianten aus "Bemerkung" alphabetisch sammeln ---

# "Bemerkung" bereinigen: Zeilenumbrüche/Mehrfachspaces -> 1 Space, trimmen
bem = df_clean["Bemerkung"].astype("string")
bem_norm = (
    bem
    .astype("string")
    .str.lower()
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

# NaNs sichtbar machen (sonst gehen sie ggf. unter)
bem_norm = bem_norm.fillna("<NA>")

# Alphabetische Liste + Counts
bem_counts = (
    bem_norm
    .value_counts(dropna=False)
    .sort_index()  # alphabetisch nach dem Wert
    .reset_index()
    .rename(columns={"index": "Bemerkung", "count": "Anzahl"})
)

display(bem_counts)
print(f"Anzahl verschiedener Varianten: {bem_counts.shape[0]}")

In [ ]:
# Optional: bereinigte Daten speichern
df_clean.to_csv("../data/processed/aufschreibung_hra_clean.csv", index=False)
df_clean.to_excel("../data/processed/aufschreibung_hra_clean.xlsx", index=False)

print("Bereinigte Daten gespeichert! :D")
